In [1]:
import gradio as gr
import numpy as np
from PIL import Image
from typing import Tuple
import io

# Assuming we have our ArtAnalyzer class defined
from artclass_class import ArtAnalyzer

def load_model():
    # Initialize our analyzer with the correct style order from your training
    style_labels = [
    "naive_art",
    "baroque",
    "rococo",
    "romanticism", 
    "art_deco",
    "american_realism",
    "art_nouveau",
    "expressionism",
    "post_impressionism",
    "high_renaissance",
    "cubism",
    "abstract_expressionism",
    "art_informel",
    "mannerism",
    "northern_renaissance",
    "surrealism",
    "symbolism", 
    "early_renaissance",
    "modernism",
    "neo_romantic",
    "ukiyo_e",
    "impressionism",
    "pop_art",
    "fauvism",
    "neoclassicism",
    "minimalism"
]
    
    return ArtAnalyzer(
    cnn_model_path="art_classifier_best.keras",
    style_labels=style_labels
)

def preprocess_image(image):
    """Helper function to handle different image formats including webp"""
    try:
        if isinstance(image, str):  # If image is a file path
            img = Image.open(image)
        elif isinstance(image, np.ndarray):  # If image is already a numpy array
            img = Image.fromarray(image)
        else:
            return None, "Unsupported image format"
            
        # Convert webp to RGB if needed
        if img.format == 'WEBP':
            img = img.convert('RGB')
            
        return img, None
            
    except Exception as e:
        return None, f"Error processing image: {str(e)}"

def process_and_analyze(image: np.ndarray, question: str) -> Tuple[str, str]:
    """
    Process image and use ArtAnalyzer's methods for analysis
    """
    try:
        # Check if image was provided
        if image is None:
            return "Error: No image provided", "Please upload an image"
            
        # Preprocess image
        img, error = preprocess_image(image)
        if error:
            return f"Error: {error}", "Unable to process image"
            
        # Resize and normalize
        img = img.resize((256, 256))
        img_array = np.array(img) / 255.0
        
        # Use analyzer's analyze_image method
        predictions = analyzer.analyze_image(img_array)
        
        # Format style predictions
        style_output = "Predicted Styles:\n"
        sorted_predictions = sorted(predictions.items(), key=lambda x: x[1], reverse=True)
        for style, conf in sorted_predictions[:3]:
            style_output += f"{style}: {conf:.2%}\n"
        
        # Use analyzer's question answering method
        response = analyzer.answer_question(question, predictions)
        if "Question:" in response:
            response = response.split("Answer:", 1)[-1].strip()
        
        return style_output, response
        
    except Exception as e:
        return f"Error: {str(e)}", "An unexpected error occurred"

# Initialize analyzer
analyzer = load_model()

# Create Gradio interface
with gr.Blocks() as demo:
    gr.Markdown("# Art Style Analyzer")
    
    with gr.Row():
        # Simplified image input
        image_input = gr.Image(
            type="numpy",
            label="Upload artwork (supported formats: jpg, png, webp)"
        )
        question_input = gr.Textbox(
            label="Ask a question about the artwork",
            placeholder="What techniques are used in this style?"
        )
    
    with gr.Row():
        style_output = gr.Textbox(label="Style Predictions")
        answer_output = gr.Textbox(label="Answer")
    
    analyze_btn = gr.Button("Analyze Artwork")
    
    analyze_btn.click(
        fn=process_and_analyze,
        inputs=[image_input, question_input],
        outputs=[style_output, answer_output]
    )
# Launch the interface
if __name__ == "__main__":
    demo.launch()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)
